# Main imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Enable interactive plots mode (NOT advised because it slows down everything down)
#%matplotlib widget
%matplotlib inline

In [3]:
# Imports 
from glob import glob
import os
# fix for ete 
os.environ['QT_QPA_PLATFORM']='offscreen'

import cv2
import numpy as np 
import sys
import scipy
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
import matplotlib.pyplot as plt
# switch   to the dark side
#plt.style.use('dark_background')
from mltools.batch_processing import open_dets

import gc
from tifffile import TiffWriter,TiffFile, imwrite,imread
import torch
import copy
from IPython.display import clear_output # this modules allows to flush output
# from tqdm import tqdm 
from tqdm.notebook import trange, tqdm
from glob import glob
sys.path.append('../utils')
from track_fns import load_tree
from fluo_extraction_fns import *


# Process an experiment

In [4]:
import cv2
from detectron2.structures import Boxes
import matplotlib.pyplot as plt
import sys
import pickle
import argparse

import numpy as np
from matplotlib import pyplot as plt
from LineageTree import lineageTree
import scipy.stats as stats
import xml.etree.ElementTree as ET
from functools import partial
from tqdm.contrib.concurrent import process_map
import pickle
from concurrent.futures import ThreadPoolExecutor
from threading import Lock
import warnings
from image_helpers import dask_img
from fluo_extraction_fns import process_det
from tqdm.auto import tqdm
import concurrent.futures
import zarr
from os.path import basename,dirname


## Load trackmate XML in a tree file (500 uM TMZ)

## Step #1 Segment all objects contained in tracks  

In [18]:
%pdb

Automatic pdb calling has been turned ON


In [35]:
xmls = ('/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_C6_control3_high.xml',
        '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_C6_non_fluorescent5.xml',
        '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_F5_TMZ_high5.xml',
        '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_F5_TMZ_low8.xml',
        '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_F6_TMZ5_high4.xml',
        '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_F6_TMZ_non_fluorscent2.xml',
        '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_C5_control_high3.xml',
       )
tifs_bf = ( '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_C6.tif',
            '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_C6.tif',
            '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_F5.tif',
            '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_F5.tif',
            '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_F6.tif',
            '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_F6.tif',
            '/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_C5.tif',
            )
assert len(xmls) == len(tifs_bf)

for xml,tif in zip(xmls,tifs_bf):
    folder = dirname(tif)
    tif_ff = os.path.join(folder,'flatfield/',basename(tif).replace('.tif','_c2_flatfield.tif'))
    tif_fluo = os.path.join(folder,basename(tif).replace('.tif','_c2.tif'))
    out_dets = os.path.join(folder,'processed/',basename(xml).replace('.xml','_dets.bin'))
    out_fluos =  os.path.join(folder,'processed/',basename(xml).replace('.xml','_fluo_dat.bin'))
    if os.path.isfile(out_fluos) and os.path.isfile(out_dets):
        continue
    assert os.path.isfile(tif_fluo), tif_fluo
    assert os.path.isfile(tif_ff), tif_ff
    assert tif_fluo != tif
    assert out_fluos != out_dets != tif != xml
    print(tif,tif_fluo,out_dets,out_fluos)  
    
    tree,root = load_tree(xml)
    imgs_bf = dask_img(tif)
    imgs = dask_img(tif_fluo)
    flatfield = dask_img(tif_ff)
    assert imgs_bf.shape == imgs.shape == flatfield.shape 
    
    # segment with SAM
    if os.path.isfile(out_dets):
        dets = open_dets(out_dets)
    else:
        dets = trackmate_track_to_detections(tree,root,imgs_bf,out_dets)
    # then extract fluorescence 
    fnames_template = None

    # Then crop and extract the cell mask,background etc.
    with open(out_fluos,'wb') as f: 
        lock = Lock()
        futures = {}
        with ThreadPoolExecutor(max_workers=25) as executor: #15 threads uses ~50% of the available memory for a 13000*11000 image
            for det in dets:
                t = det.coords['t']
                futures[ executor.submit(process_det,det,imgs[t],imgs_bf[t],f,lock,flatfield=flatfield[t])] = t 
            for future in tqdm(concurrent.futures.as_completed(futures)):
                continue
    # run sequentially (for debugging)
    # with open(out_fluos,'wb') as f: 
    #     lock = Lock()   
    #     for det in dets:
    #         t = det.coords['t']
    #         process_det(det,imgs[t],imgs_bf[t],f,lock,flatfield=flatfield[t])

/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_C5.tif /mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/well_C5_c2.tif /mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/processed/well_C5_control_high3_dets.bin /mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/processed/well_C5_control_high3_fluo_dat.bin
time step: 60.0 min pixel size: [1. 1.] um


  0%|          | 0/75 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [16]:
ret = extract_reg_from_det_single_masks(det,imgs[det.coords['t']],imgs_bf[det.coords['t']],flatfield[det.coords['t']])

In [70]:
# run detection and save it save the detection objects
out_bin = '/mnt/dioscuri-nas/Ilyas/Results/Cells/54PD_wellC6_all_dets.bin'
dets = trackmate_track_to_detections(tree,root,imgs_bf,out_bin)

  0%|          | 0/74 [00:00<?, ?it/s]

In [13]:
dets = open_dets('/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/processed/well_F5_TMZ_high5_dets.bin')

In [14]:
# Save to ROIs for a quick check
rois= save_to_ImageJ_ROIs(dets,'/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/processed/well_F5_TMZ_high5_dets.zip',categories=None)

{}


## Step #2: crop and extract fluorescence around the cell

In [71]:
fnames_template = None #'/mnt/dioscuri-nas/Ilyas/Stitched_wells/36PD_timelapse_red_only/well_15/img_t{:04d}_z1_c1'
with open('/mnt/dioscuri-nas/Ilyas/Results/Cells/54PD_wellC6_all_cells_imgs_w_fit_bg_new.bin','wb') as f: 
    lock = Lock()
    futures = {}
    with ThreadPoolExecutor(max_workers=25) as executor: #15 threads uses ~50% of the available memory for a 13000*11000 image
        for det in dets:
            t = det.coords['t']
            futures[ executor.submit(process_det,det,imgs[t],imgs_bf[t],f,lock,flatfield=False)] = t 
        for future in tqdm(concurrent.futures.as_completed(futures)):
            continue


0it [00:00, ?it/s]

In [ ]:
## We cam quickly check the images

In [9]:
dat = load_from_pickle('/mnt/dioscuri-nas/Ilyas/Stitched_wells/54PD_downscaled/processed/well_F5_TMZ_high5_fluo_dat.bin',None)
crops = [ e['crop'] for e in dat.values()]
crops = [np.where(e.mask,0,e) for e in crops] 
imwrite('/tmp/54pd_F5_highcells.tif',crops)

Ran out of input


# Bonus: find the coordinates of the region Pragyesh cropped ( we need them to get the red channel)...

In [16]:
from utils.image_helpers import alignImages

In [18]:
# Read reference image
# imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
imReference = imread('/mnt/dioscuri-nas/Ilyas/Stitched_wells/40PD_U87MG_mgmt-OFP_cells_with_filtered_media_BF_only_split_timepoints2/Cells_with_100ul_media_2/well_4/img_t0_z1_c1')
im = dask.array.from_zarr(imread('/mnt/dioscuri-nas/Ilyas/Stitched_wells/40PD_U87MG_mgmt-OFP_cells_with_filtered_media_BF_only_split_timepoints2/well_4_corrected_crop.tif',aszarr=True))[0].compute()

print("Aligning images ...")
# Registered image will be resotred in imReg.
# The estimated homography will be stored in h.
h,mask = alignImages((im/256).astype(np.uint8), (imReference/256).astype(np.uint8))


NameError: name 'da' is not defined

In [19]:
print("Aligning images ...")
# Registered image will be resotred in imReg.
# The estimated homography will be stored in h.
h,mask = alignImages((im/256).astype(np.uint8), (imReference/256).astype(np.uint8))


Aligning images ...


In [21]:
# for well 4
h,im.shape

(array([[ 1.00001463e+00,  2.91372522e-05,  3.81588233e+03],
        [-3.03654248e-06,  1.00005596e+00,  5.68798869e+03],
        [-1.07259627e-10,  3.54794082e-09,  1.00000000e+00]]),
 (10920, 11088))

In [206]:
# for welll 7
h,img.shape

(array([[9.99994191e-01, 2.03591964e-06, 2.78400620e+03],
        [2.01566904e-07, 1.00000339e+00, 3.74399545e+03],
        [3.18555259e-11, 2.25708153e-10, 1.00000000e+00]]),
 (11136, 13088))

In [19]:
from utils.fluo_extraction_fns import some_function

In [23]:
some_function()

bar3
